In [2]:
# https://keras.io/
!pip install -q keras
import keras
!pip install h5py pyyaml 

Using TensorFlow backend.


In [3]:
import tensorflow as tf
tf.test.gpu_device_name()


'/device:GPU:0'

In [0]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.5 GB  | Proc size: 749.9 MB
GPU RAM Free: 11320MB | Used: 121MB | Util   1% | Total 11441MB


In [0]:
!kill -9 -1

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 250
l = 38
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [7]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 20s 0us/step


In [0]:
def normalize(x):
    min_val = np.min(x)
    max_val = np.max(x)
    
    x = (x-min_val)/(max_val-min_val)
    return x

In [0]:
x_train = normalize(x_train)
x_test = normalize(x_test)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    cv = Conv2D(10, (1,1), use_bias=False ,padding='same')(relu)
    avg = AveragePooling2D(pool_size=(2,2))(cv)
    pooling = GlobalAveragePooling2D()(avg)
    output = Activation('softmax')(pooling)
    #output = Dense(num_classes, activation='softmax')(pooling)
    
    return output

In [13]:
num_filter = 12
dropout_rate = 0.2
l=36

input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(16, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


Instructions for updating:
Colocations handled automatically by placer.


In [14]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
from keras.callbacks import LearningRateScheduler
def step_lr(epoch):
    initial_lr = 0.1
    if epoch>=90:
        learning_rate = initial_lr/10
    if epoch>=135:
        learning_rate = initial_lr/100
    if epoch<90:
        learning_rate = initial_lr
    return learning_rate

lrate = LearningRateScheduler(step_lr)

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.1, decay=1e-4, momentum=0.9, nesterov=True),
              metrics=['accuracy'])


In [0]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    #zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=30,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.2,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.2,
    #shear_range=0.2,  # set range for random shear(init-0)
    #zoom_range=0.2,  # set range for random zoom(init-0)
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

In [0]:
class Logger(keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs={}):
        print('current_learning_rate:',k.eval(self.model.optimizer.lr))
  

lr_log = Logger()

In [18]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Colab Notebooks/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
checkpoint_path = "/content/gdrive/My Drive/densenet_training/cp1-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True, period=10)

In [0]:
!ls gdrive



'My Drive'


In [0]:
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    epochs=epochs,steps_per_epoch=x_train.shape[0]//batch_size,
                    validation_data=(x_test, y_test),
                    validation_steps = x_test.shape[0]//batch_size,
                    #use_multiprocessing=True,
                    callbacks=[ lrate, lr_log, cp_callback])

Instructions for updating:
Use tf.cast instead.
Epoch 1/250
781/781 [==============================] - 686s 878ms/step - loss: 1.8104 - acc: 0.3296 - val_loss: 1.6286 - val_acc: 0.4038
current_learning_rate: 0.1
Epoch 2/250
781/781 [==============================] - 643s 823ms/step - loss: 1.5078 - acc: 0.4435 - val_loss: 2.2161 - val_acc: 0.3551
current_learning_rate: 0.1
Epoch 3/250
781/781 [==============================] - 642s 822ms/step - loss: 1.3413 - acc: 0.5116 - val_loss: 1.3574 - val_acc: 0.5300
current_learning_rate: 0.1
Epoch 4/250
781/781 [==============================] - 642s 822ms/step - loss: 1.2095 - acc: 0.5592 - val_loss: 1.2202 - val_acc: 0.5696
current_learning_rate: 0.1
Epoch 5/250
781/781 [==============================] - 640s 820ms/step - loss: 1.0936 - acc: 0.6071 - val_loss: 1.7269 - val_acc: 0.4986
current_learning_rate: 0.1
Epoch 6/250
781/781 [==============================] - 639s 818ms/step - loss: 1.0128 - acc: 0.6391 - val_loss: 0.9576 - val_acc: 0.

In [0]:
! ls {checkpoint_dir}

/device:GPU:0


In [0]:
model.load_weights('/content/gdrive/My Drive/densenet_training/cp1-0050.ckpt')

In [0]:
#Training after 50th epoch
epochs=200
from keras.callbacks import LearningRateScheduler
def step_lr(epoch):
    initial_lr = 0.1
    if epoch>=40:
        learning_rate = initial_lr/10
    if epoch>=85:
        learning_rate = initial_lr/100
    if epoch<40:
        learning_rate = initial_lr
    return learning_rate

lrate = LearningRateScheduler(step_lr)


In [0]:
#checkpoints after 50th epoch
import os
checkpoint_path = "/content/gdrive/My Drive/densenet_training/cp1-50-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True, period=3)

In [0]:
#Resume training 
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    epochs=epochs,steps_per_epoch=x_train.shape[0]//batch_size,
                    validation_data=(x_test, y_test),
                    validation_steps = x_test.shape[0]//batch_size,
                    
                    #use_multiprocessing=True,
                    callbacks=[ lrate, lr_log, cp_callback])

Epoch 1/200
781/781 [==============================] - 660s 845ms/step - loss: 0.4625 - acc: 0.8372 - val_loss: 0.7676 - val_acc: 0.7670
current_learning_rate: 0.1
Epoch 2/200
781/781 [==============================] - 661s 846ms/step - loss: 0.4563 - acc: 0.8416 - val_loss: 0.4950 - val_acc: 0.8343
current_learning_rate: 0.1
Epoch 3/200
781/781 [==============================] - 661s 847ms/step - loss: 0.4385 - acc: 0.8476 - val_loss: 0.5117 - val_acc: 0.8283
current_learning_rate: 0.1

Epoch 00003: saving model to /content/gdrive/My Drive/densenet_training/cp1-50-0003.ckpt
Epoch 4/200
781/781 [==============================] - 661s 846ms/step - loss: 0.4295 - acc: 0.8497 - val_loss: 0.5208 - val_acc: 0.8281
current_learning_rate: 0.1
Epoch 5/200
781/781 [==============================] - 661s 846ms/step - loss: 0.4231 - acc: 0.8529 - val_loss: 0.5685 - val_acc: 0.8174
current_learning_rate: 0.1
Epoch 6/200
781/781 [==============================] - 661s 847ms/step - loss: 0.4126 - ac

In [0]:
#Training after 83rd epoch
epochs=167
from keras.callbacks import LearningRateScheduler
def step_lr(epoch):
    initial_lr = 0.1
    if epoch>=7:
        learning_rate = initial_lr/10
    if epoch>=52:
        learning_rate = initial_lr/100
    if epoch<7:
        learning_rate = initial_lr
    return learning_rate

lrate = LearningRateScheduler(step_lr)


In [0]:
#checkpoints after 83rd epoch
import os
checkpoint_path = "/content/gdrive/My Drive/densenet_training/cp1-83-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True, period=3)

In [0]:
#loading the model
model.load_weights('/content/gdrive/My Drive/densenet_training/cp1-50-0033.ckpt')

In [0]:
#Resume training 
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    epochs=epochs,steps_per_epoch=x_train.shape[0]//batch_size,
                    validation_data=(x_test, y_test),
                    validation_steps = x_test.shape[0]//batch_size,
                    
                    #use_multiprocessing=True,
                    callbacks=[ lrate, lr_log, cp_callback])

Instructions for updating:
Use tf.cast instead.
Epoch 1/167
781/781 [==============================] - 669s 856ms/step - loss: 0.4410 - acc: 0.8470 - val_loss: 0.6139 - val_acc: 0.8016
current_learning_rate: 0.1
Epoch 2/167
781/781 [==============================] - 642s 823ms/step - loss: 0.4193 - acc: 0.8548 - val_loss: 0.7854 - val_acc: 0.7757
current_learning_rate: 0.1
Epoch 3/167
781/781 [==============================] - 642s 821ms/step - loss: 0.3966 - acc: 0.8619 - val_loss: 0.5448 - val_acc: 0.8228
current_learning_rate: 0.1

Epoch 00003: saving model to /content/gdrive/My Drive/densenet_training/cp1-83-0003.ckpt
Epoch 4/167
781/781 [==============================] - 643s 823ms/step - loss: 0.3788 - acc: 0.8677 - val_loss: 0.5342 - val_acc: 0.8291
current_learning_rate: 0.1
Epoch 5/167
781/781 [==============================] - 643s 823ms/step - loss: 0.3718 - acc: 0.8712 - val_loss: 0.5025 - val_acc: 0.8379
current_learning_rate: 0.1
Epoch 6/167
781/781 [=====================

In [0]:
import os
checkpoint_path = "/content/gdrive/My Drive/densenet_training/cp1-146-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True, period=10)

In [0]:
#Training after 146th epoch
epochs=104
from keras.callbacks import LearningRateScheduler
def step_lr(epoch):
    initial_lr = 0.001
    if epoch>=20:
        learning_rate = initial_lr/10
    else:
        learning_rate = initial_lr
    return learning_rate

lrate = LearningRateScheduler(step_lr)


In [0]:
#loading the model
model.load_weights('/content/gdrive/My Drive/densenet_training/cp1-83-0063.ckpt')

In [25]:
#Resume training 
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    epochs=epochs,steps_per_epoch=x_train.shape[0]//batch_size,
                    validation_data=(x_test, y_test),
                    validation_steps = x_test.shape[0]//batch_size,
                    
                    #use_multiprocessing=True,
                    callbacks=[ lrate, lr_log, cp_callback])

Epoch 1/104
781/781 [==============================] - 680s 871ms/step - loss: 0.1953 - acc: 0.9311 - val_loss: 0.3443 - val_acc: 0.8901
current_learning_rate: 0.001
Epoch 2/104
781/781 [==============================] - 656s 840ms/step - loss: 0.1945 - acc: 0.9317 - val_loss: 0.3381 - val_acc: 0.8920
current_learning_rate: 0.001
Epoch 3/104
781/781 [==============================] - 656s 839ms/step - loss: 0.1996 - acc: 0.9301 - val_loss: 0.3438 - val_acc: 0.8910
current_learning_rate: 0.001
Epoch 4/104
781/781 [==============================] - 655s 839ms/step - loss: 0.1964 - acc: 0.9313 - val_loss: 0.3469 - val_acc: 0.8902
current_learning_rate: 0.001
Epoch 5/104
781/781 [==============================] - 654s 837ms/step - loss: 0.1984 - acc: 0.9297 - val_loss: 0.3451 - val_acc: 0.8886
current_learning_rate: 0.001
Epoch 6/104
781/781 [==============================] - 654s 838ms/step - loss: 0.1965 - acc: 0.9309 - val_loss: 0.3417 - val_acc: 0.8903
current_learning_rate: 0.001
Epoc

KeyboardInterrupt: ignored

In [26]:
#Resume training 
batch_size = 128
epochs = 104-28
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    epochs=epochs,steps_per_epoch=x_train.shape[0]//batch_size,
                    validation_data=(x_test, y_test),
                    validation_steps = x_test.shape[0]//batch_size,
                    
                    #use_multiprocessing=True,
                    callbacks=[ lrate, lr_log, cp_callback])

Epoch 1/76
390/390 [==============================] - 558s 1s/step - loss: 0.1828 - acc: 0.9358 - val_loss: 0.3418 - val_acc: 0.8931
current_learning_rate: 0.001
Epoch 2/76
390/390 [==============================] - 556s 1s/step - loss: 0.1849 - acc: 0.9360 - val_loss: 0.3408 - val_acc: 0.8923
current_learning_rate: 0.001
Epoch 3/76
390/390 [==============================] - 551s 1s/step - loss: 0.1815 - acc: 0.9365 - val_loss: 0.3429 - val_acc: 0.8913
current_learning_rate: 0.001

Epoch 00003: saving model to /content/gdrive/My Drive/densenet_training/cp1-146-0003.ckpt
Epoch 4/76
390/390 [==============================] - 551s 1s/step - loss: 0.1821 - acc: 0.9346 - val_loss: 0.3426 - val_acc: 0.8914
current_learning_rate: 0.001
Epoch 5/76
390/390 [==============================] - 552s 1s/step - loss: 0.1839 - acc: 0.9362 - val_loss: 0.3419 - val_acc: 0.8917
current_learning_rate: 0.001
Epoch 6/76
 16/390 [>.............................] - ETA: 8:05 - loss: 0.1811 - acc: 0.9321

KeyboardInterrupt: ignored

In [0]:
#Training after 146th epoch
epochs=104-28-6
from keras.callbacks import LearningRateScheduler
def step_lr(epoch):
    learning_rate = 0.0001
#     if epoch>=20:
#         learning_rate = initial_lr/10
#     else:
#         learning_rate = initial_lr
    return learning_rate

lrate = LearningRateScheduler(step_lr)


In [28]:
#Resume training 
batch_size = 128
epochs = 104-28
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    epochs=epochs,steps_per_epoch=x_train.shape[0]//batch_size,
                    validation_data=(x_test, y_test),
                    validation_steps = x_test.shape[0]//batch_size,
                    
                    #use_multiprocessing=True,
                    callbacks=[ lrate, lr_log, cp_callback])

Epoch 1/76
390/390 [==============================] - 552s 1s/step - loss: 0.1842 - acc: 0.9352 - val_loss: 0.3421 - val_acc: 0.8913
current_learning_rate: 1e-04
Epoch 2/76
390/390 [==============================] - 552s 1s/step - loss: 0.1818 - acc: 0.9360 - val_loss: 0.3403 - val_acc: 0.8916
current_learning_rate: 1e-04
Epoch 3/76
390/390 [==============================] - 552s 1s/step - loss: 0.1825 - acc: 0.9364 - val_loss: 0.3430 - val_acc: 0.8909
current_learning_rate: 1e-04
Epoch 4/76
390/390 [==============================] - 552s 1s/step - loss: 0.1819 - acc: 0.9359 - val_loss: 0.3421 - val_acc: 0.8914
current_learning_rate: 1e-04
Epoch 5/76
390/390 [==============================] - 553s 1s/step - loss: 0.1824 - acc: 0.9364 - val_loss: 0.3410 - val_acc: 0.8915
current_learning_rate: 1e-04
Epoch 6/76
390/390 [==============================] - 553s 1s/step - loss: 0.1855 - acc: 0.9346 - val_loss: 0.3424 - val_acc: 0.8911
current_learning_rate: 1e-04
Epoch 7/76
390/390 [========

KeyboardInterrupt: ignored

In [0]:
#Resume training 
batch_size = 32
epochs = 104-28
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    epochs=epochs,steps_per_epoch=x_train.shape[0]//batch_size,
                    validation_data=(x_test, y_test),
                    validation_steps = x_test.shape[0]//batch_size,
                    
                    #use_multiprocessing=True,
                    callbacks=[ lrate, lr_log, cp_callback])

Epoch 1/76
1562/1562 [==============================] - 821s 526ms/step - loss: 0.2185 - acc: 0.9244 - val_loss: 0.3384 - val_acc: 0.8924
current_learning_rate: 1e-04
Epoch 2/76
1562/1562 [==============================] - 823s 527ms/step - loss: 0.2200 - acc: 0.9246 - val_loss: 0.3408 - val_acc: 0.8919
current_learning_rate: 1e-04

Epoch 00002: saving model to /content/gdrive/My Drive/densenet_training/cp1-146-0002.ckpt
Epoch 3/76
1562/1562 [==============================] - 821s 526ms/step - loss: 0.2181 - acc: 0.9231 - val_loss: 0.3467 - val_acc: 0.8912
current_learning_rate: 1e-04
Epoch 4/76
1562/1562 [==============================] - 825s 528ms/step - loss: 0.2128 - acc: 0.9259 - val_loss: 0.3438 - val_acc: 0.8917
current_learning_rate: 1e-04
Epoch 5/76
1562/1562 [==============================] - 822s 526ms/step - loss: 0.2168 - acc: 0.9249 - val_loss: 0.3440 - val_acc: 0.8921
current_learning_rate: 1e-04
Epoch 6/76
1562/1562 [==============================] - 818s 524ms/step - 

In [22]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 48s 5ms/step
Test loss: 0.3436819578289986
Test accuracy: 0.8919


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model_sota1(77).h5")
print("Saved model to disk")

Saved model to disk


In [0]:
model.save_weights("DNST_model_sota1(119).h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model_sota1(119).h5')